# CRQ2
###### Visualize Taxis movements! NYC is divided in many Taxis zones. For each yellow cab trip we know the zone the Taxi pick up and drop off the users. Let's visualize, on a chropleth map, the number of trips that starts in each zone. Than, do another map to count the races that end up in the single zone. Comment your discoveries. To perform this task we use the library folium. 

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from sys import getsizeof as get
import folium
import json
import branca.colormap as cm

In [4]:
M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-01.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-02.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-03.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-04.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-05.csv", encoding = 'latin-1')
#M1 = pd.read_csv(r"C:\Users\Egon\Desktop\Universita\ADM\homework2\dati\yellow_tripdata_2018-06.csv", encoding = 'latin-1')

In [5]:
M1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


We need twocolumns. So, we drop the others. keep PULocationID, DOLocationID

In [6]:
#dropping vs keeping the ones I need?
#df.drop(columns=['', 'C'])
M2 = M1.filter(['DOLocationID', 'PULocationID'])
#free the memory from this big object M1 is! 
del M1

In [8]:
M2.head()

,DOLocationID,PULocationID
0,24,41
1,140,239
2,141,262
3,257,140
4,239,246


At first we want a map with Picking locations. 

In [11]:
pickup=M2.filter(['DOLocationID'])

In [23]:
pickupCAT=pickup.

count    8.759874e+06
mean     1.627269e+02
std      7.031164e+01
min      1.000000e+00
25%      1.130000e+02
50%      1.620000e+02
75%      2.340000e+02
max      2.650000e+02
Name: DOLocationID, dtype: float64

In [37]:

folium.Map(
    location=[41.9161422, 12.5600524],
    tiles='Stamen Terrain',
    zoom_start=13
)
